### Cumulative stats up to the year before to predict the following years salary 

### Regression - linear, decision, neural networks

### Classification - extend contract or release player

In [1]:
import requests
import json
import pandas as pd
import pickle
import numpy as np

pd.set_option('display.max_columns', 999)

In [2]:
nhl_teams = {}

for team in range(1,102):
    
    url = f"https://statsapi.web.nhl.com/api/v1/teams/{team}"

    headers = {
        'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
        'x-rapidapi-host': "api-hockey.p.rapidapi.com"
    }

    nhl_teams[team] = requests.request("GET", url, headers=headers)
    
    with open(f'json_files/nhl_teams/teams_{team}', 'wb') as f:
        pickle.dump(f'json_files/nhl_teams/teams_{team}', f)

In [3]:
errors_404 = [59, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 84, 85, 86, 99]

In [4]:
for i in errors_404:
    nhl_teams.pop(i)

In [5]:
def get_teams(team):
    '''This function takes in a dictionary, 'nhl_teams', from the NHL API created by 
    the requests library and returns a pandas dataframe of all the teams in the NHL (active & inactive)'''   

    # Initialize an empty dictionary
    teams = {}
    # Allow for iteration over the index values       
    for i in nhl_teams.keys():
        # Initialize a sub dictionary so data is not overwritten
        teams[i] = {}
        # Create a for loop to 
        for keys in team[i].json()['teams'][0].keys():
            
            teams[i][keys] = team[i].json()['teams'][0][keys]

            nested_dict = ['venue', 'division', 'conference', 'franchise']

            for nested in nested_dict:

                if keys == nested:

                    for value in team[i].json()['teams'][0][keys].keys():

                        teams[i][f'{nested}_{value}'] = team[i].json()['teams'][0][keys][value]

                        if value == 'timeZone':

                            teams[i][f'venue_{value}'] = team[i].json()['teams'][0][keys][value]['id']
    
    teams_df = pd.DataFrame(teams, columns = teams.keys()).T
    
    teams_df.drop(nested_dict, axis=1, inplace = True)
    
    return teams_df

In [6]:
all_teams_df = get_teams(nhl_teams)
all_teams_df.head()

,id,name,link,venue_name,venue_link,venue_city,venue_timeZone,abbreviation,teamName,locationName,firstYearOfPlay,division_id,division_name,division_link,conference_id,conference_name,conference_link,franchise_franchiseId,franchise_teamName,franchise_link,shortName,officialSiteUrl,franchiseId,active,venue_id,division_nameShort,division_abbreviation
1,1,New Jersey Devils,/api/v1/teams/1,Prudential Center,/api/v1/venues/null,Newark,America/New_York,NJD,Devils,New Jersey,1982,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,23,Devils,/api/v1/franchises/23,New Jersey,http://www.newjerseydevils.com/,23,True,NaN,NaN,NaN
2,2,New York Islanders,/api/v1/teams/2,Nassau Veterans Memorial Coliseum,/api/v1/venues/null,Uniondale,America/New_York,NYI,Islanders,New York,1972,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,22,Islanders,/api/v1/franchises/22,NY Islanders,http://www.newyorkislanders.com/,22,True,NaN,NaN,NaN
3,3,New York Rangers,/api/v1/teams/3,Madison Square Garden,/api/v1/venues/5054,New York,America/New_York,NYR,Rangers,New York,1926,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,10,Rangers,/api/v1/franchises/10,NY Rangers,http://www.newyorkrangers.com/,10,True,5054,NaN,NaN
4,4,Philadelphia Flyers,/api/v1/teams/4,Wells Fargo Center,/api/v1/venues/5096,Philadelphia,America/New_York,PHI,Flyers,Philadelphia,1967,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,16,Flyers,/api/v1/franchises/16,Philadelphia,http://www.philadelphiaflyers.com/,16,True,5096,NaN,NaN
5,5,Pittsburgh Penguins,/api/v1/teams/5,PPG Paints Arena,/api/v1/venues/5034,Pittsburgh,America/New_York,PIT,Penguins,Pittsburgh,1967,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,17,Penguins,/api/v1/franchises/17,Pittsburgh,http://pittsburghpenguins.com/,17,True,5034,NaN,NaN


In [7]:
nhl_team_stadiums = {}

for link in all_teams_df['venue_link']:

    url = f"https://statsapi.web.nhl.com/api/v1/teams/{link}/roster"

    headers = {
        'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
        'x-rapidapi-host': "api-hockey.p.rapidapi.com"
    }

    nhl_team_stadiums[link] = requests.request("GET", url, headers=headers)

In [8]:
nhl_team_stadiums

{'/api/v1/venues/null': <Response [404]>,
 '/api/v1/venues/5054': <Response [404]>,
 '/api/v1/venues/5096': <Response [404]>,
 '/api/v1/venues/5034': <Response [404]>,
 '/api/v1/venues/5085': <Response [404]>,
 '/api/v1/venues/5039': <Response [404]>,
 '/api/v1/venues/5028': <Response [404]>,
 '/api/v1/venues/5031': <Response [404]>,
 '/api/v1/venues/5066': <Response [404]>,
 '/api/v1/venues/5027': <Response [404]>,
 '/api/v1/venues/5094': <Response [404]>,
 '/api/v1/venues/5092': <Response [404]>,
 '/api/v1/venues/5145': <Response [404]>,
 '/api/v1/venues/5030': <Response [404]>,
 '/api/v1/venues/5076': <Response [404]>,
 '/api/v1/venues/5075': <Response [404]>,
 '/api/v1/venues/5064': <Response [404]>,
 '/api/v1/venues/5100': <Response [404]>,
 '/api/v1/venues/5073': <Response [404]>,
 '/api/v1/venues/5046': <Response [404]>,
 '/api/v1/venues/5019': <Response [404]>,
 '/api/v1/venues/5081': <Response [404]>,
 '/api/v1/venues/5059': <Response [404]>,
 '/api/v1/venues/5098': <Response 

In [9]:
nhl_team_franchise = {}

for link in all_teams_df['franchise_link']:

    url = f"https://statsapi.web.nhl.com/api/v1/teams/{link}/roster"

    headers = {
        'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
        'x-rapidapi-host': "api-hockey.p.rapidapi.com"
    }

    nhl_team_franchise[link] = requests.request("GET", url, headers=headers)

In [10]:
nhl_team_franchDataFrame

{'/api/v1/franchises/23': <Response [404]>,
 '/api/v1/franchises/22': <Response [404]>,
 '/api/v1/franchises/10': <Response [404]>,
 '/api/v1/franchises/16': <Response [404]>,
 '/api/v1/franchises/17': <Response [404]>,
 '/api/v1/franchises/6': <Response [404]>,
 '/api/v1/franchises/19': <Response [404]>,
 '/api/v1/franchises/1': <Response [404]>,
 '/api/v1/franchises/30': <Response [404]>,
 '/api/v1/franchises/5': <Response [404]>,
 '/api/v1/franchises/35': <Response [404]>,
 '/api/v1/franchises/26': <Response [404]>,
 '/api/v1/franchises/33': <Response [404]>,
 '/api/v1/franchises/31': <Response [404]>,
 '/api/v1/franchises/24': <Response [404]>,
 '/api/v1/franchises/11': <Response [404]>,
 '/api/v1/franchises/12': <Response [404]>,
 '/api/v1/franchises/34': <Response [404]>,
 '/api/v1/franchises/18': <Response [404]>,
 '/api/v1/franchises/21': <Response [404]>,
 '/api/v1/franchises/27': <Response [404]>,
 '/api/v1/franchises/25': <Response [404]>,
 '/api/v1/franchises/20': <Response

In [11]:
nhl_team_roster = {}

for number in range(1,55):

    url = f"https://statsapi.web.nhl.com/api/v1/teams/{number}/roster"

    headers = {
        'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
        'x-rapidapi-host': "api-hockey.p.rapidapi.com"
    }

    nhl_team_roster[number] = requests.request("GET", url, headers=headers)

In [ ]:
nhl_team_roster

In [12]:
roster_error_404 = [11, 27, 31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]
for error in roster_error_404:
    nhl_team_roster.pop(error)

In [13]:
def build_roster(roster):
    '''This function takes in a dictionary, 'nhl_team_roster', from the NHL API created
    by the requests library and returns a pandas dataframe of all players with associated
    team number based from the api'''
    
    # Initialize empy dictionary
    person_dict = {}
    # Allow for iteration over the index values (API team number)        
    for num in roster.keys():
        # Initialize a sub dictionary so data is not overwritten
        person_dict[num] = {}
        # Create a for loop that will loop over every teams roster due to varying amounts of total players                                
        for player_index in range(0, len(roster[num].json()['roster'])):
            # Initialize a sub dictionary so data is not overwritten on a team level 
            person_dict[num][player_index] = {}
            # Create a for loop that will loop over all of the keys within the 'roster' nested dictionaries
            for key in roster[num].json()['roster'][player_index].keys():
                # Allow the function to iterate over the 'person' nested dictionary                         
                if key == 'person':
                    # Looping over the 'person' nested dictionary
                    for person in roster[num].json()['roster'][player_index][key]:
                        # Adding values to the dictionary
                        person_dict[num][player_index][person] = roster[num].json()['roster'][player_index][key][person]
                # Allow the function to iterate over the 'position' nested dictionary
                if key == 'position':
                    # Looping over the 'position' nested dictionary
                    for position in roster[num].json()['roster'][player_index][key]:
                        # Adding values to the dictionary
                        person_dict[num][player_index][position] = roster[num].json()['roster'][player_index][key][position]
                # Allow the function to add the jersey number to the dictionary
                if key == 'jerseyNumber':
                    # Adding value to the dictionary
                    person_dict[num][player_index][key] = roster[num].json()['roster'][player_index][key]
        
    # Adding the API's team number as a value within the nested dictionaries in case we need to join on this value at a later time
    for value in person_dict:
        
        for length in range(0, len(person_dict[value])):
        
            person_dict[value][length]['Team_Number'] = value
        
    # Initializing an empty list to convert into a pandas dataframe
    df_list = []
    # Iterating over the above created dictionary
    for num in person_dict:
        # Create a for loop that will loop over every teams roster due to varying amounts of total players
        for length in range(0, len(person_dict[num])):
            # Appending values to dictionary
            df_list.append(person_dict[num][length])
    # Creating Pandas DataFrame
    roster_df = pd.DataFrame(df_list)  
        
    
    return roster_df
        

In [14]:
rosters = build_roster(nhl_team_roster)

In [15]:
rosters

,id,fullName,link,jerseyNumber,code,name,type,abbreviation,Team_Number
0,8477972,Josh Jacobs,/api/v1/people/8477972,40,D,Defenseman,Defenseman,D,1
1,8478421,A.J. Greer,/api/v1/people/8478421,42,L,Left Wing,Forward,LW,1
2,8474056,P.K. Subban,/api/v1/people/8474056,76,D,Defenseman,Defenseman,D,1
3,8475809,Scott Wedgewood,/api/v1/people/8475809,41,G,Goalie,Goalie,G,1
4,8476807,Matt Tennyson,/api/v1/people/8476807,7,D,Defenseman,Defenseman,D,1
...,...,...,...,...,...,...,...,...,...
945,8479639,Dylan Coghlan,/api/v1/people/8479639,52,D,Defenseman,Defenseman,D,54
946,8479980,Nicolas Hague,/api/v1/people/8479980,14,D,Defenseman,Defenseman,D,54
947,8479996,Cody Glass,/api/v1/people/8479996,9,C,Center,Forward,C,54
948,8480313,Logan Thompson,/api/v1/people/8480313,36,G,Goalie,Goalie,G,54


In [16]:
player_stats = {}
for link in rosters['link']:

    url = f"https://statsapi.web.nhl.com/{link}"

    headers = {
        'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
        'x-rapidapi-host': "api-hockey.p.rapidapi.com"
    }

    player_stats[link] = requests.request("GET", url, headers=headers)

In [17]:
def player_statistics(df, player_stat):
    
    player_dict = {}
    
    for link in df['link']:
        
        player_dict[link] = {}
        
        for info in player_stat[link].json()['people'][0].keys():
            
            player_dict[link][info] = player_stats[link].json()['people'][0][info]
            
            if info == 'currentTeam':
                
                for team in player_stats[link].json()['people'][0][info]:
                    
                    player_dict[link][f'current_team_{team}'] = player_stats[link].json()['people'][0][info][team]
                    
            if info == 'primaryPosition':
                
                for position in player_stats[link].json()['people'][0][info]:
                
                    player_dict[link][f'primary_position_{position}'] = player_stats[link].json()['people'][0][info][position]
                 
    player_df = pd.DataFrame(player_dict).T.reset_index()
    
    player_df.drop(['currentTeam', 'primaryPosition'], axis = 1, inplace = True)
    
    return player_df       

In [18]:
df = player_statistics(rosters, player_stats)

In [19]:
full_player_df = pd.concat([rosters, df], axis = 1, join='inner')

In [20]:
full_player_df

,id,fullName,link,jerseyNumber,code,name,type,abbreviation,Team_Number,index,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus,current_team_id,current_team_name,current_team_link,primary_position_code,primary_position_name,primary_position_type,primary_position_abbreviation
0,8477972,Josh Jacobs,/api/v1/people/8477972,40,D,Defenseman,Defenseman,D,1,/api/v1/people/8477972,8477972,Josh Jacobs,/api/v1/people/8477972,Josh,Jacobs,40,1996-02-15,25,Shelby Township,MI,USA,USA,"6' 2""",200,True,False,False,True,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
1,8478421,A.J. Greer,/api/v1/people/8478421,42,L,Left Wing,Forward,LW,1,/api/v1/people/8478421,8478421,A.J. Greer,/api/v1/people/8478421,A.J.,Greer,42,1996-12-14,24,Joliette,QC,CAN,CAN,"6' 3""",210,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,L,Left Wing,Forward,LW
2,8474056,P.K. Subban,/api/v1/people/8474056,76,D,Defenseman,Defenseman,D,1,/api/v1/people/8474056,8474056,P.K. Subban,/api/v1/people/8474056,P.K.,Subban,76,1989-05-13,32,Toronto,ON,CAN,CAN,"6' 0""",210,True,True,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
3,8475809,Scott Wedgewood,/api/v1/people/8475809,41,G,Goalie,Goalie,G,1,/api/v1/people/8475809,8475809,Scott Wedgewood,/api/v1/people/8475809,Scott,Wedgewood,41,1992-08-14,28,Brampton,ON,CAN,CAN,"6' 2""",207,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,G,Goalie,Goalie,G
4,8476807,Matt Tennyson,/api/v1/people/8476807,7,D,Defenseman,Defenseman,D,1,/api/v1/people/8476807,8476807,Matt Tennyson,/api/v1/people/8476807,Matt,Tennyson,7,1990-04-23,31,Minneapolis,MN,USA,USA,"6' 2""",205,True,False,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,8479639,Dylan Coghlan,/api/v1/people/8479639,52,D,Defenseman,Defenseman,D,54,/api/v1/people/8479639,8479639,Dylan Coghlan,/api/v1/people/8479639,Dylan,Coghlan,52,1998-02-19,23,Duncan,BC,CAN,CAN,"6' 2""",206,True,False,False,True,R,Y,54,Vegas Golden Knights,/api/v1/teams/54,D,Defenseman,Defenseman,D
946,8479980,Nicolas Hague,/api/v1/people/8479980,14,D,Defenseman,Defenseman,D,54,/api/v1/people/8479980,8479980,Nicolas Hague,/api/v1/people/8479980,Nicolas,Hague,14,1998-12-05,22,Kitchener,ON,CAN,CAN,"6' 6""",230,True,False,False,False,L,Y,54,Vegas Golden Knights,/api/v1/teams/54,D,Defenseman,Defenseman,D
947,8479996,Cody Glass,/api/v1/people/8479996,9,C,Center,Forward,C,54,/api/v1/people/8479996,8479996,Cody Glass,/api/v1/people/8479996,Cody,Glass,9,1999-04-01,22,Winnipeg,MB,CAN,CAN,"6' 3""",206,True,False,False,False,R,Y,54,Vegas Golden Knights,/api/v1/teams/54,C,Center,Forward,C
948,8480313,Logan Thompson,/api/v1/people/8480313,36,G,Goalie,Goalie,G,54,/api/v1/people/8480313,8480313,Logan Thompson,/api/v1/people/8480313,Logan,Thompson,36,1997-02-25,24,Calgary,AB,CAN,CAN,"6' 4""",201,True,False,False,True,R,Y,54,Vegas Golden Knights,/api/v1/teams/54,G,Goalie,Goalie,G


In [21]:
# count_teams = 0

# for i in rosters:
#     for n in rosters[i]:
#         count += n - (n-1)
# print(count_teams)

# count_roster = 0

# for i in rosters:
#     count += i - (i-1)
    
# print(count_roster)

In [22]:
player_stats_20192020 = {}

years = ['20192020']

for link in rosters['link']:
    
    for val in years:

        url = f'https://statsapi.web.nhl.com/{link}/stats?stats=statsSingleSeason&season={val}'

        headers = {
            'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
            'x-rapidapi-host': "api-hockey.p.rapidapi.com"
        }

        player_stats_20192020[link] = requests.request("GET", url, headers=headers)

In [23]:
player_stats_20192020

{'/api/v1/people/8477972': <Response [200]>,
 '/api/v1/people/8478421': <Response [200]>,
 '/api/v1/people/8474056': <Response [200]>,
 '/api/v1/people/8475809': <Response [200]>,
 '/api/v1/people/8476807': <Response [200]>,
 '/api/v1/people/8476850': <Response [200]>,
 '/api/v1/people/8476923': <Response [200]>,
 '/api/v1/people/8476941': <Response [200]>,
 '/api/v1/people/8477180': <Response [200]>,
 '/api/v1/people/8477341': <Response [200]>,
 '/api/v1/people/8477355': <Response [200]>,
 '/api/v1/people/8477425': <Response [200]>,
 '/api/v1/people/8478399': <Response [200]>,
 '/api/v1/people/8478401': <Response [200]>,
 '/api/v1/people/8478406': <Response [200]>,
 '/api/v1/people/8478447': <Response [200]>,
 '/api/v1/people/8479407': <Response [200]>,
 '/api/v1/people/8479414': <Response [200]>,
 '/api/v1/people/8479415': <Response [200]>,
 '/api/v1/people/8479511': <Response [200]>,
 '/api/v1/people/8479573': <Response [200]>,
 '/api/v1/people/8480002': <Response [200]>,
 '/api/v1/

In [24]:
player_stats_20182019 = {}

years = ['20182019']

for link in rosters['link']:
    
    for val in years:

        url = f'https://statsapi.web.nhl.com/{link}/stats?stats=statsSingleSeason&season={val}'

        headers = {
            'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
            'x-rapidapi-host': "api-hockey.p.rapidapi.com"
        }

        player_stats_20182019[link] = requests.request("GET", url, headers=headers)

In [25]:
player_stats_20172018 = {}

years = ['20172018']

for link in rosters['link']:
    
    for val in years:

        url = f'https://statsapi.web.nhl.com/{link}/stats?stats=statsSingleSeason&season={val}'

        headers = {
            'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
            'x-rapidapi-host': "api-hockey.p.rapidapi.com"
        }

        player_stats_20172018[link] = requests.request("GET", url, headers=headers)

In [26]:
player_stats_20162017 = {}

years = ['20162017']

for link in rosters['link']:
    
    for val in years:

        url = f'https://statsapi.web.nhl.com/{link}/stats?stats=statsSingleSeason&season={val}'

        headers = {
            'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
            'x-rapidapi-host': "api-hockey.p.rapidapi.com"
        }

        player_stats_20162017[link] = requests.request("GET", url, headers=headers)

In [27]:
player_stats_20152016 = {}

years = ['20152016']

for link in rosters['link']:
    
    for val in years:

        url = f'https://statsapi.web.nhl.com/{link}/stats?stats=statsSingleSeason&season={val}'

        headers = {
            'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
            'x-rapidapi-host': "api-hockey.p.rapidapi.com"
        }

        player_stats_20152016[link] = requests.request("GET", url, headers=headers)

In [28]:
player_stats_20142015 = {}

years = ['20142015']

for link in rosters['link']:
    
    for val in years:

        url = f'https://statsapi.web.nhl.com/{link}/stats?stats=statsSingleSeason&season={val}'

        headers = {
            'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
            'x-rapidapi-host': "api-hockey.p.rapidapi.com"
        }

        player_stats_20142015[link] = requests.request("GET", url, headers=headers)

In [29]:
player_stats_20132014 = {}

years = ['20132014']

for link in rosters['link']:
    
    for val in years:

        url = f'https://statsapi.web.nhl.com/{link}/stats?stats=statsSingleSeason&season={val}'

        headers = {
            'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
            'x-rapidapi-host': "api-hockey.p.rapidapi.com"
        }

        player_stats_20132014[link] = requests.request("GET", url, headers=headers)

In [30]:
  '20192020', '20182019', '20172018', '20162017', '20152016', '20142015', '20132014', '20122013', '20112012', '20102011'\
        '2009/2010', '20082009', '20072008', '20062007', '20052006', '20042005', '20042004', '20032004', '20022003', '20012002', '2000,2001'

('20192020',
 '20182019',
 '20172018',
 '20162017',
 '20152016',
 '20142015',
 '20132014',
 '20122013',
 '20112012',
 '201020112009/2010',
 '20082009',
 '20072008',
 '20062007',
 '20052006',
 '20042005',
 '20042004',
 '20032004',
 '20022003',
 '20012002',
 '2000,2001')